In [ ]:
#Import libraries
import os
import requests

from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
from openai import OpenAI 
import google.generative.ai 
import anthropic